# 05.01 - Modeling Setup - Custom Model Classes

## Imports & setup

In [1]:
import pathlib
import warnings
from datetime import datetime
import math
import sys

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

sys.path.append("..")

from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, RegressorMixin
from statsmodels.tsa.statespace.sarimax import SARIMAX
from fbprophet import Prophet

%matplotlib inline

PROJECT_DIR = pathlib.Path.cwd().parent.resolve()
CLEAN_DATA_DIR = PROJECT_DIR / 'data' / '05-clean'

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Load Some Data for Testing

In [2]:
df = pd.read_csv(CLEAN_DATA_DIR / 'clean-cut.csv', parse_dates=True, index_col=0)
df = df.loc['1994': '1995']
df = df.resample('D').max()
# Just select a reasonable subset of data to test the model wrappers
df = df[['temp', 'dew_point_temp', 'week_of_year', 'daily_peak']]
df.rename(columns={'temp': 'temp_max'}, inplace=True)

y = df.pop('daily_peak')
X = df

X.head()

,temp_max,dew_point_temp,week_of_year
1994-01-01,2.8,1.1,52.0
1994-01-02,1.7,0.5,52.0
1994-01-03,-10.3,-12.6,1.0
1994-01-04,-7.4,-11.5,1.0
1994-01-05,-7.2,-10.7,1.0


In [3]:
y.head()

1994-01-01    16892.0
1994-01-02    18947.0
1994-01-03    21923.0
1994-01-04    21457.0
1994-01-05    22082.0
Freq: D, Name: daily_peak, dtype: float64

## Baseline Model

In [4]:
class SetTempAsPower:
    """
    Makes a forecast by scaling temperature by the training set power min and max
    Returns the scaled temperature as a prediction
    Additional method get_pred_values for residual diagnostic purposes
    on last fitted and predicted model
    """

    def __init__(self, col="temp_max"):
        self.col = col

    def fit(self, X, y):
        self.y_fit = y
        self.X_fit = X
        self.max_power = y.max()
        self.min_power = y.min()
        return self

    def predict(self, X):
        self.X_predict = X
        minmaxscaler = MinMaxScaler(feature_range=(self.min_power, self.max_power))
        minmaxscaler.fit(self.X_fit[[self.col]])
        scaled = minmaxscaler.transform(self.X_predict[[self.col]])
        self.predict_yhat = pd.Series(
            data=scaled.reshape(1, -1)[0], index=self.X_predict.index
        )
        return self.predict_yhat

    def get_pred_values(self):
        # All Data is only available after fit and predict
        # Build a return DataFrame that looks similar to the prophet output
        # date index | y | yhat| yhat_lower | yhat_upper | is_forecast
        X_fit = self.X_fit.copy()
        X_predict = self.X_predict.copy()
        y = self.y_fit
        if self.X_fit.equals(self.X_predict):
            yhat = self.predict_yhat.copy(deep=True)

        else:
            self.fit(X_fit, y)
            y.name = "y"
            fit_yhat = self.predict(X_fit)
            pred_yhat = self.predict(X_predict)
            y_pred = pd.Series(np.NaN, index=X_predict.index)
            y = pd.concat([y, y_pred], axis=0)
            yhat = pd.concat([fit_yhat, pred_yhat], axis=0)

        yhat.name = "yhat"
        y.name = "y"
        y = self.y_fit.copy()
        full_suite = pd.concat([yhat, y], axis=1)
        full_suite["is_forecast"] = 0
        full_suite["is_forecast"] = full_suite["y"].isna().astype(int)
        full_suite["yhat_upper"] = np.NaN
        full_suite["yhat_lower"] = np.NaN

        full_suite = full_suite[
            ["y", "yhat", "yhat_lower", "yhat_upper", "is_forecast"]
        ]

        return full_suite

In [5]:
X_m = X.copy(deep=True)
y_m = y.copy(deep=True)
X_train = X_m['1994'] ; y_train = y_m['1994']
X_test = X_m['1995'] ; y_test = y_m['1995']

set_temp_as_power = SetTempAsPower(col='temp_max')
set_temp_as_power.fit(X_train, y_train)
preds = set_temp_as_power.predict(X_test)
print(preds)
print()
print(mean_absolute_error(y_test, preds))

1995-01-01    17382.451087
1995-01-02    17019.626812
1995-01-03    16446.746377
1995-01-04    15511.041667
1995-01-05    15740.193841
                  ...     
1995-12-27    16103.018116
1995-12-28    16561.322464
1995-12-29    16733.186594
1995-12-30    16885.954710
1995-12-31    17325.163043
Freq: D, Length: 365, dtype: float64

3465.6429720071465


## SARIMAX Model

In [6]:
class SK_SARIMAX(BaseEstimator, RegressorMixin):
    """
    Wrapper for StatsModels SARIMAX tp a Scikit Learn Style Regressor
    Additional Method get_pred_values to compile a dataset for the last model run
    and return it for diangnostics purposes
    """
    
    def __init__(self, order=(2,0,1), seasonal_order=(2,0,0,96), trend='c'):
        self.order = order
        self.seasonal_order=seasonal_order
        self.trend=trend

    def fit(self, X, y):
        self.fit_X = X
        self.fit_y = y
        self.model = SARIMAX(self.fit_y,
                            order=self.order,
                            seasonal_order=self.seasonal_order,
                            trend=self.trend,
                            exog=self.fit_X)
        self.results = self.model.fit(disp=False)
        return self.model
        
    def predict(self, X, y=None):
        self.predict_X = X
        self.forecast_object = self.results.get_forecast(steps=len(X), exog=X)
        self.conf_int = self.forecast_object.conf_int()
        self.ser = pd.Series(data = self.forecast_object.predicted_mean.values, index = self.predict_X.index)
        return self.ser
    
    def get_pred_values(self):
        # All Data is only available after fit and predict
        # Build a return DataFrame that looks similar to the prophet output
        # date index y | yhat| yhat_lower | yhat_upper | is_forecast
        fitted = self.fit_y.copy(deep=True)
        fitted.name = 'y'
        fitted = pd.DataFrame(fitted)
        fitted['is_forecast'] = 0
        fitted['yhat'] = self.results.predict()
        
        ser = self.ser.copy(deep=True)
        predict_y = pd.DataFrame(self.ser, columns=['yhat'])
        predict_y['is_forecast'] = 1

        conf_ints = pd.DataFrame(self.forecast_object.conf_int().values,
                                 index = predict_y.index, columns=['yhat_lower', 'yhat_upper'])
        
        unknown = pd.concat([predict_y, conf_ints], axis=1, sort=True)

        full_suite = pd.concat([fitted, unknown], axis=0, sort=True)
        full_suite = full_suite[['y', 'yhat', 'yhat_lower', 'yhat_upper', 'is_forecast']]

        return full_suite

In [7]:
X_m = X.copy(deep=True)
y_m = y.copy(deep=True)
X_train = X_m['1994'] ; y_train = y_m['1994']
X_test = X_m['1995'] ; y_test = y_m['1995']

sk_sarimax = SK_SARIMAX(order=(1,1,1), seasonal_order=(0,0,0,96), trend='c')
sk_sarimax.fit(X_train, y_train)
preds = sk_sarimax.predict(X_test)
print(preds)
print()
print(mean_absolute_error(y_test, preds))

1995-01-01    17740.355312
1995-01-02    19384.757528
1995-01-03    19409.174761
1995-01-04    19599.444730
1995-01-05    19526.055538
                  ...     
1995-12-27    16779.312783
1995-12-28    16696.124261
1995-12-29    16665.023338
1995-12-30    16640.312896
1995-12-31    16569.806737
Freq: D, Length: 365, dtype: float64

1663.7301177460001


## Prophet Model

In [8]:
from sklearn.base import BaseEstimator, RegressorMixin
from fbprophet import Prophet

class SK_Prophet(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    
    def __init__(self, regressors={}, pred_periods=96):
#         self.pred_periods=pred_periods
        self.regressors=regressors
       
        
    def prep_X(self, X):
        # Prophet requires a DataFrame with a column of dates labeled 'ds'
        if 'ds' not in X.columns:
            X = X.assign(ds = X.index)
            X.reset_index(drop=True, inplace=True)
        return X
    
    def prep_y(self, y):
        # prohet requires the target to be labeled as y
        if y.name is not 'y':
            y.name = 'y'
        return y
        
    def fit(self, X, y):
        self.X_fit = X.copy(deep=True)
        self.y_fit = y.copy(deep=True)
        
        # Setup the model
        self.model=Prophet(daily_seasonality=True)
        self.model.add_seasonality(name='summer', period=96, fourier_order=5)
        self.model.add_seasonality(name='workweek', period=5, fourier_order=5)
        self.model.seasonality_mode= 'multiplicative'

        # If regressors is not empty
        if (self.regressors.keys()):
            for regressor, params in self.regressors.items():
                if params:
                    self.model.add_regressor(regressor, prior_scale=params[0], mode=params[1])
                else:
                    self.model.add_regressor(regressor)
        
        # Setup the data
        X = self.prep_X(X)
        y = self.prep_y(y)       
        df = X.merge(right=y, left_on='ds', right_on=y.index)
        
        self.model.fit(df)
        return self.model
        
    def predict(self, X):
        self.X_pred = X.copy(deep=True)
        X_ = self.prep_X(self.X_pred)
        forecast_object = self.model.predict(X_)
        # When we return the prediction, we are looking to return a datetime indexed series
        # Therefore, we need to reverse what was done in prep_X prior to returning
        preds = forecast_object.copy()
        preds.set_index('ds', drop=True, inplace=True)
        preds = preds['yhat']
        preds.index.names=['date']
        return preds
    
    def get_pred_values(self):
        # All Data is only available after fit and predict
        # Build a return DataFrame that looks similar to the prophet output
        # date index y | yhat| yhat_lower | yhat_upper | is_forecast
        full_X = pd.concat([self.prep_X(self.X_fit),
                            self.prep_X(self.X_pred)],
                            axis=0).reset_index(drop=True)
        forecast_obj = self.model.predict(full_X)
        forecast_obj['is_forecast'] = 0
        forecast_obj.set_index('ds',drop=True, inplace=True)
        del forecast_obj.index.name
        forecast_obj.loc[self.y_fit.index, 'y'] = self.y_fit.values
        forecast_obj.loc[self.X_pred.index, 'is_forecast'] = 1
        
        return forecast_obj

In [9]:
X_m = X.copy(deep=True)
y_m = y.copy(deep=True)
X_train = X_m['1994'] ; y_train = y_m['1994']
X_test = X_m['1995'] ; y_test = y_m['1995']

prophet_model = SK_Prophet(regressors={'temp_max':(1.0, 'additive')})
prophet_model.fit(X_train, y_train)
preds = prophet_model.predict(X_test)
print(preds)
print()
print(mean_absolute_error(y_test, preds))

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/home/david/anaconda3/envs/ana_py37tf/lib/python3.7/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



date
1995-01-01    17267.793802
1995-01-02    19271.690028
1995-01-03    19448.960799
1995-01-04    19800.554209
1995-01-05    19657.770584
                  ...     
1995-12-27    28575.271794
1995-12-28    28415.174914
1995-12-29    27781.930797
1995-12-30    25872.928642
1995-12-31    25286.884568
Name: yhat, Length: 365, dtype: float64

4754.444363021953


## XGBoost Model

$$residual\;error = actual - predicted$$

$$enhanced\;prediction = predicted + predicted\;residual\;error$$  

In [10]:
X_m = X.copy(deep=True)
y_m = y.copy(deep=True)
X_train = X_m['1994'] ; y_train = y_m['1994']
X_test = X_m['1995'] ; y_test = y_m['1995']

set_temp_as_power = SetTempAsPower(col='temp_max')
set_temp_as_power.fit(X_train, y_train)
orig_preds = set_temp_as_power.predict(X_test)
resid_data = set_temp_as_power.get_pred_values()
print(resid_data)

                  y          yhat  yhat_lower  yhat_upper  is_forecast
1994-01-01  16892.0  17668.891304         NaN         NaN            0
1994-01-02  18947.0  17458.835145         NaN         NaN            0
1994-01-03  21923.0  15167.313406         NaN         NaN            0
1994-01-04  21457.0  15721.097826         NaN         NaN            0
1994-01-05  22082.0  15759.289855         NaN         NaN            0
...             ...           ...         ...         ...          ...
1995-12-27      NaN  16103.018116         NaN         NaN            1
1995-12-28      NaN  16561.322464         NaN         NaN            1
1995-12-29      NaN  16733.186594         NaN         NaN            1
1995-12-30      NaN  16885.954710         NaN         NaN            1
1995-12-31      NaN  17325.163043         NaN         NaN            1

[730 rows x 5 columns]


In [11]:
resid_data.loc['1995', 'y'] = y_test.values
print(resid_data)

                  y          yhat  yhat_lower  yhat_upper  is_forecast
1994-01-01  16892.0  17668.891304         NaN         NaN            0
1994-01-02  18947.0  17458.835145         NaN         NaN            0
1994-01-03  21923.0  15167.313406         NaN         NaN            0
1994-01-04  21457.0  15721.097826         NaN         NaN            0
1994-01-05  22082.0  15759.289855         NaN         NaN            0
...             ...           ...         ...         ...          ...
1995-12-27  19260.0  16103.018116         NaN         NaN            1
1995-12-28  19014.0  16561.322464         NaN         NaN            1
1995-12-29  18635.0  16733.186594         NaN         NaN            1
1995-12-30  18132.0  16885.954710         NaN         NaN            1
1995-12-31  17333.0  17325.163043         NaN         NaN            1

[730 rows x 5 columns]


In [12]:
resids = resid_data['y'].subtract(resid_data['yhat'])
print(resids)

1994-01-01    -776.891304
1994-01-02    1488.164855
1994-01-03    6755.686594
1994-01-04    5735.902174
1994-01-05    6322.710145
                 ...     
1995-12-27    3156.981884
1995-12-28    2452.677536
1995-12-29    1901.813406
1995-12-30    1246.045290
1995-12-31       7.836957
Freq: D, Length: 730, dtype: float64


In [13]:
resids_1994 = resids.loc['1994']
resids_1995 = resids.loc['1995']


from xgboost.sklearn import XGBRegressor

xgboost = XGBRegressor(max_depth=5,objective='reg:squarederror', n_estimators=200, learning_rate=0.1,
                      importance_type='gain')

xgboost.fit(X_train, resids_1994)
xg_preds = xgboost.predict(X_test)
full_preds = orig_preds.add(xg_preds)
full_preds

/home/david/anaconda3/envs/ana_py37tf/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/home/david/anaconda3/envs/ana_py37tf/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



1995-01-01    19123.227698
1995-01-02    20930.004741
1995-01-03    20035.595498
1995-01-04    21380.522135
1995-01-05    21856.418938
                  ...     
1995-12-27    21019.140675
1995-12-28    19431.099075
1995-12-29    19327.892649
1995-12-30    19152.566527
1995-12-31    19080.712116
Freq: D, Length: 365, dtype: float64

In [14]:
mean_absolute_error(y_test, full_preds)

1065.0773549563266

Improved from 3465